In [77]:
import pandas as pd
def to_dataframe(texts):
    ids = []
    labels = []
    parents = []
    info = []
    levels = []
    label_length = []

    for idx, text in enumerate(texts):
        parts = text.split('-')
        for i, part in enumerate(parts):
            ids.append("-".join(parts[:i+1]))
            labels.append(part)
            parents.append("-".join(parts[:i]) if i > 0 else "")
            info.append(part)
            levels.append(i+1)
            label_length.append(len(part))

            
            
    return pd.DataFrame({
        "ids": ids,
        "labels": labels,
        "parents": parents,
        "info": info,
        "level": levels,
        "label_length": label_length,
    })
    
# 函数功能：将文本转换为dataframe


In [82]:
with open('example.txt', 'r') as f:
    example = f.read()

input_list = example.split('\n')
# print(input_list)
input_list = to_dataframe(input_list)
# print(input_list)

In [62]:
# 打印df的标题
print(input_list.columns)

Index(['ids', 'labels', 'parents', 'info', 'level', 'label_length'], dtype='object')


In [65]:
# 答应df的标题
print(input_list.columns)
# 打印df.length
print(input_list.label_length)

Index(['ids', 'labels', 'parents', 'info', 'level', 'label_length'], dtype='object')
0       8
1      40
2     321
3       7
4      41
5     403
6      10
7      38
8     392
9       7
10     45
11    138
12      7
13     33
14    126
15      7
16     40
17    102
18      7
19     33
20     75
21      7
22     33
23    162
Name: label_length, dtype: int64


In [75]:
import textwrap
def auto_wrap_text(text, max_width):
    wrapped_text = textwrap.wrap(text, max_width)
    wrapped_text = "<br>".join(wrapped_text)
    return wrapped_text

def apply_wrap(row):
    if row['level'] == 1:
        return auto_wrap_text(row['labels'], max_width=10)
    elif row['level'] == 2:
        return auto_wrap_text(row['labels'], max_width=20)
    else:
        return auto_wrap_text(row['labels'], max_width=20)


# 尝试不同的内容的换行
# 不同level的换行

In [83]:
import random
import colorsys

base_hue = random.random()  # 在Python的colorsys模块中，色调的范围是0-1，所以我们需要把角度除以360

# 生成色系相似的8个颜色
base_colors = []
for i in range(8):
    # 在色调上加上一个小的偏移（在这里是i*10度），并确保结果在0-1范围内
    hue = (base_hue + i*10/360) % 1
    # 使用饱和度1和亮度0.5生成颜色（这将生成一个鲜艳的颜色）
    color = colorsys.hls_to_rgb(hue, 0.5, 1)
    # 将颜色从RGB空间转换到十六进制颜色代码
    base_colors.append("#"+''.join([f"{int(val*255):02x}" for val in color]))
    

# 转换十六进制颜色代码为RGB颜色值（范围是0-1）
base_colors_rgb = [[int(color[i:i+2], 16) / 255 for i in range(1, 6, 2)] for color in base_colors]

# 节点和层级的关系，假设我们有8个段落，每个段落有3个层级
nodes = ["P{}-L{}".format(p, l) for p in range(8) for l in range(3)]
levels = [l for p in range(8) for l in range(3)]

# 根据节点的层级和对应的基础颜色来生成颜色
node_colors = []
for i in range(8):
    for j in range(3):
        # 随着层级增加，将颜色的亮度增加
        h, l, s = colorsys.rgb_to_hls(*base_colors_rgb[i])
        color = colorsys.hls_to_rgb(h, min(1, l + 0.2*j), s)  # 每次增加20%的亮度，最大为1
        node_colors.append("rgba"+str(tuple([int(255*val) for val in color] + [1.0])) + ")")

In [92]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import pandas as pd
import json
import textwrap
import numpy as np
import plotly.colors as colors
import matplotlib.pyplot as plt
import random
import colorsys



# 读取文本
with open('example.txt', 'r') as f:
    example = f.read()
input_list = example.split('\n')
input_list = to_dataframe(input_list)


# 生成模拟数据
df = pd.DataFrame(input_list)
df["labels"] = df.apply(apply_wrap, axis=1)



fig = go.Figure(go.Treemap(
    ids=df['ids'],
    labels=df['labels'],
    parents=df['parents'],

    maxdepth=2,
    # text=df['labels'],
    insidetextfont=dict(size=30, color='#FFFFFF'),
    outsidetextfont=dict(size=15, color='#FF0000', family='Arial'),
    # marker=dict(
    #     colors=df['label_length'],  # 使用label_length列的值来定义颜色
    #     colorscale='Viridis'  # 定义颜色比例尺,"Twilight, Viridis"、"Cividis"、"Plasma"
    # ),
    marker=dict(
        colors=df['label_length'],  # 使用label_length列的值来定义颜色
        colorscale='Reds',  # 定义自定义颜色比例尺，从红色到浅红色
        reversescale=True  # 颜色比例尺反转，从红色到蓝色
    ),
    # marker_colors=node_colors,  # 使用标签长度作为权重
    
    tiling=dict(
    packing='squarify', # slice-dice  # Set packing algorithm to 'squarify' # 布局方式
    pad=0.2
    )
))

fig.update_layout(
    # title="人工智能在不同行业的应用",  # 添加标题
    title={
    'text': "<b>人工智能在不同行业的应用",
    'x': 0.5,  # 将标题的 x 位置设置为 0.5，即图形的中心
    'xanchor': 'center',  # 将标题的 x 锚点设置为 'center'，即标题的中心
    # 'y' : 0.96 , # 高度调整
    },
    title_font=dict(
    family="Courier New, monospace",  # 设置字体
    size=20,  # 设置字体大小
    color="RebeccaPurple"  # 设置字体颜色
    ),
    # uniformtext=dict(minsize=15, mode='hide'), 
    margin = dict(t=50, l=25, r=25, b=25),
    width=600,
    height=600,

)


fig.update_traces(marker=dict(cornerradius=20)) # 圆角
fig.show()


